In [2]:
import numpy as np
import cv2
import os
import shutil
from google.colab import files

In [3]:
face_cascade = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('./haarcascade_eye.xml')

In [4]:
def get_cropped_image_if_2_eyes(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        if len(eyes) >= 2:
            return roi_color

In [5]:
path_to_data = "./dataset/"
path_to_cr_data = "./dataset/cropped/"

In [12]:
img_dirs = []
for entry in os.scandir(path_to_data):
    if entry.is_dir():
        img_dirs.append(entry.path)

In [13]:
img_dirs

['./dataset/Warner', './dataset/Maxwell', './dataset/Cummins']

In [14]:
if os.path.exists(path_to_cr_data):
     shutil.rmtree(path_to_cr_data)
os.mkdir(path_to_cr_data)

In [15]:
# Function to detect and crop the face if two eyes are detected
def get_cropped_image_if_2_eyes(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Unable to read image {image_path}")
        return None

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)

        if len(eyes) >= 2:
            roi_color = img[y:y+h, x:x+w]
            return roi_color
    return None

# Main code to process images
cropped_image_dirs = []
celebrity_file_names_dict = {}

for img_dir in img_dirs:
    count = 1
    celebrity_name = img_dir.split('/')[-1]
    print(celebrity_name)

    celebrity_file_names_dict[celebrity_name] = []

    for entry in os.scandir(img_dir):
        if entry.is_file():  # Ensure the entry is a file
            roi_color = get_cropped_image_if_2_eyes(entry.path)
            if roi_color is not None:
                cropped_folder = os.path.join(path_to_cr_data, celebrity_name)
                if not os.path.exists(cropped_folder):
                    os.makedirs(cropped_folder)
                    cropped_image_dirs.append(cropped_folder)
                    print("Generating cropped images in folder:", cropped_folder)

                cropped_file_name = f"{celebrity_name}{count}.png"
                cropped_file_path = os.path.join(cropped_folder, cropped_file_name)

                cv2.imwrite(cropped_file_path, roi_color)
                celebrity_file_names_dict[celebrity_name].append(cropped_file_path)
                count += 1

Warner
Generating cropped images in folder: ./dataset/cropped/Warner
Maxwell
Generating cropped images in folder: ./dataset/cropped/Maxwell
Error: Unable to read image ./dataset/Maxwell/10633.gif
Error: Unable to read image ./dataset/Maxwell/10228.svg
Cummins
Generating cropped images in folder: ./dataset/cropped/Cummins


In [16]:
# Define the path to your cropped folder
cropped_folder_path = path_to_cr_data  # Update this to your actual cropped folder path

# Define the name for the zip file
zip_file_name = 'cropped_images.zip'

# Zip the cropped folder
shutil.make_archive(zip_file_name.replace('.zip', ''), 'zip', cropped_folder_path)

# Download the zip file
files.download(zip_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>